# Tool selection example

When having too many tools, we can use a tool selection mechanism to choose the most appropriate tool for the task at hand.

## Manual approach

In [1]:
import rich
from agentic_patterns.core.agents import get_agent, run_agent

### Create tools

In [2]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    print(f"Adding {a} + {b}")
    return a + b


def sub(a: int, b: int) -> int:
    """Subtract two numbers"""
    print(f"Subtracting {a} - {b}")
    return a - b

In [3]:
tools = [add, sub]
tools_by_name = {tool.__name__: tool for tool in tools}

In [4]:
def func2descr(f):
    out = ""
    # Add the signature
    out += f"Tool: {f.__name__}({', '.join(f.__code__.co_varnames[: f.__code__.co_argcount])})\n"
    # Add the docstring
    if f.__doc__:
        out += f"Description: {f.__doc__}\n"
    # Add the return type
    if f.__annotations__ and "return" in f.__annotations__:
        out += f"Return type: {f.__annotations__['return']}\n"
    return out

In [5]:
print(func2descr(add))

Tool: add(a, b)
Description: Add two numbers
Return type: <class 'int'>



In [6]:
tools_descriptions = [func2descr(tool) for tool in tools]
tools_descriptions_str = "\n".join(tools_descriptions)
print(tools_descriptions_str)

Tool: add(a, b)
Description: Add two numbers
Return type: <class 'int'>

Tool: sub(a, b)
Description: Subtract two numbers
Return type: <class 'int'>



### Select tools

In [7]:
agent = get_agent(output_type=list[str])

user_query = "What is the sum of 40123456789 and 2123456789?"

prompt = f"""
Given the user query, select the tools to use

User query: {user_query}

Tools available:
{tools_descriptions_str}

Answer only using the tool names
"""
result, _ = await run_agent(agent, prompt, verbose=True)
print(result.result.output)

Agent step: UserPromptNode(user_prompt="\nGiven the user query, select the tools to use\n\nUser query: What is the 
sum of 40123456789 and 2123456789?\n\nTools available:\nTool: add(a, b)\nDescription: Add two numbers\nReturn type:
<class 'int'>\n\nTool: sub(a, b)\nDescription: Subtract two numbers\nReturn type: <class 'int'>\n\n\nAnswer only 
using the tool names\n", instructions_functions=[], system_prompts=(), system_prompt_functions=[], 
system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content="\nGiven the user query, select the
tools to use\n\nUser query: What is the sum of 40123456789 and 2123456789?\n\nTools available:\nTool: add(a, 
b)\nDescription: Add two numbers\nReturn type: <class 'int'>\n\nTool: sub(a, b)\nDescription: Subtract two 
numbers\nReturn type: <class 'int'>\n\n\nAnswer only using the tool names\n", timestamp=datetime.datetime(2026, 2, 
12, 13, 52, 56, 585119, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='final_result', 
args='{"response": ["add"]}', tool_call_id='toolu_vrtx_018TMCPygEfBcAEvoTnS3tnv')], 
usage=RequestUsage(input_tokens=763, output_tokens=40, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 52, 58, 762926, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'tool_calls', 'timestamp': datetime.datetime(2026, 2, 12, 13, 52, 57, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904376-qqbmhh5Vj3pBLcKQ1OId', finish_reason='tool_call', 
run_id='a23a5c0d-7109-4a40-bd75-8e152f4f1b54'))

Agent step: End(data=FinalResult(output=['add'], tool_name='final_result', 
tool_call_id='toolu_vrtx_018TMCPygEfBcAEvoTnS3tnv'))

['add']


In [8]:
tool_names = result.result.output
print(f"Selected tools: {tool_names}")

Selected tools: ['add']


### Use the tools

In [9]:
# Only keep the tools that were selected
tools_agent = [tools_by_name[name] for name in tool_names if name in tools_by_name]  # type: ignore

# Create a new agent with the selected tools
agent = get_agent(tools=tools_agent)

prompt = user_query
result, nodes = await run_agent(agent, prompt, verbose=True)
print(result.result.output)

Agent step: UserPromptNode(user_prompt='What is the sum of 40123456789 and 2123456789?', instructions_functions=[],
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is the sum of 40123456789 and
2123456789?', timestamp=datetime.datetime(2026, 2, 12, 13, 52, 58, 895596, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="I need to add these two numbers 
together. However, I notice that the numbers you've provided (40123456789 and 2123456789) are very large and exceed
the typical integer range that the available function might handle.\n\nLet me try to use the add function:"), 
ToolCallPart(tool_name='add', args='{"a": 40123456789, "b": 2123456789}', 
tool_call_id='toolu_vrtx_01JfyRm235dQdYMHgAtLch7z')], usage=RequestUsage(input_tokens=590, output_tokens=133, 
details={'is_byok': False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', 
timestamp=datetime.datetime(2026, 2, 12, 13, 53, 1, 368023, tzinfo=datetime.timezone.utc), provider_name='openai', 
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'tool_calls', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 52, 58, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904378-6odHGuiEPN23yJfqn9Y3', finish_reason='tool_call', 
run_id='ec2195dc-7ab0-4139-bf17-c5428782b073'))

Adding 40123456789 + 2123456789


Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='add', content=42246913578, 
tool_call_id='toolu_vrtx_01JfyRm235dQdYMHgAtLch7z', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 1, 371304, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The sum of 40123456789 and 
2123456789 is **42246913578**.')], usage=RequestUsage(input_tokens=739, output_tokens=26, details={'is_byok': 
False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 
13, 53, 2, 517680, tzinfo=datetime.timezone.utc), provider_name='openai', 
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'stop', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 53, 1, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904381-bScRY7wKBd8NNHsXQiLF', finish_reason='stop', 
run_id='ec2195dc-7ab0-4139-bf17-c5428782b073'))

Agent step: End(data=FinalResult(output='The sum of 40123456789 and 2123456789 is **42246913578**.'))

The sum of 40123456789 and 2123456789 is **42246913578**.


In [10]:
rich.print(nodes)

[
    UserPromptNode(user_prompt='What is the sum of 40123456789 and 2123456789?', instructions_functions=[], 
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={}),
    ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is the sum of 40123456789 and 
2123456789?', timestamp=datetime.datetime(2026, 2, 12, 13, 52, 58, 895596, tzinfo=datetime.timezone.utc))], 
timestamp=datetime.datetime(2026, 2, 12, 13, 52, 58, 897354, tzinfo=datetime.timezone.utc), 
run_id='ec2195dc-7ab0-4139-bf17-c5428782b073')),
    CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="I need to add these two numbers together. 
However, I notice that the numbers you've provided (40123456789 and 2123456789) are very large and exceed the 
typical integer range that the available function might handle.\n\nLet me try to use the add function:"), 
ToolCallPart(tool_name='add', args='{"a": 40123456789, "b": 2123456789}', 
tool_call_id='toolu_vrtx_01JfyRm235dQdYMHgAtLch7z')], usage=RequestUsage(input_tokens=590, output_tokens=133, 
details={'is_byok': False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', 
timestamp=datetime.datetime(2026, 2, 12, 13, 53, 1, 368023, tzinfo=datetime.timezone.utc), provider_name='openai', 
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'tool_calls', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 52, 58, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904378-6odHGuiEPN23yJfqn9Y3', finish_reason='tool_call', 
run_id='ec2195dc-7ab0-4139-bf17-c5428782b073')),
    ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='add', content=42246913578, 
tool_call_id='toolu_vrtx_01JfyRm235dQdYMHgAtLch7z', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 1, 371304, 
tzinfo=datetime.timezone.utc))], timestamp=datetime.datetime(2026, 2, 12, 13, 53, 1, 372934, 
tzinfo=datetime.timezone.utc), run_id='ec2195dc-7ab0-4139-bf17-c5428782b073')),
    CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The sum of 40123456789 and 2123456789 is 
**42246913578**.')], usage=RequestUsage(input_tokens=739, output_tokens=26, details={'is_byok': False, 
'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53,
2, 517680, tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'stop', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 1, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904381-bScRY7wKBd8NNHsXQiLF', finish_reason='stop', 
run_id='ec2195dc-7ab0-4139-bf17-c5428782b073')),
    End(data=FinalResult(output='The sum of 40123456789 and 2123456789 is **42246913578**.'))
]

## Using ToolSelector

In [11]:
from agentic_patterns.core.tools import func_to_description, ToolSelector

# Show the improved function description
print(func_to_description(add))

Tool: add(a: int, b: int) -> int
Description: Add two numbers


In [12]:
# Select tools
selector = ToolSelector([add, sub])
selected_tools = await selector.select(user_query, verbose=True)
selected_tools

Agent step: UserPromptNode(user_prompt='Given the user query, select the tools to use.\n\nUser query: What is the 
sum of 40123456789 and 2123456789?\n\nTools available:\nTool: add(a: int, b: int) -> int\nDescription: Add two 
numbers\n\nTool: sub(a: int, b: int) -> int\nDescription: Subtract two numbers\n\nAnswer only using the tool 
names.', instructions_functions=[], system_prompts=(), system_prompt_functions=[], 
system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Given the user query, select the 
tools to use.\n\nUser query: What is the sum of 40123456789 and 2123456789?\n\nTools available:\nTool: add(a: int, 
b: int) -> int\nDescription: Add two numbers\n\nTool: sub(a: int, b: int) -> int\nDescription: Subtract two 
numbers\n\nAnswer only using the tool names.', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 2, 578596, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='final_result', 
args='{"response": ["add"]}', tool_call_id='toolu_vrtx_01SZVmMfLYHBQz8c8CP12qoP')], 
usage=RequestUsage(input_tokens=755, output_tokens=40, details={'is_byok': False, 'reasoning_tokens': 0}), 
model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 3, 780052, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'tool_calls', 'timestamp': datetime.datetime(2026, 2, 12, 13, 53, 2, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904382-tJPMzjeMXCXhavZB81ub', finish_reason='tool_call', 
run_id='ca0b373d-5d88-4894-b892-6d796ac1576c'))

Agent step: End(data=FinalResult(output=['add'], tool_name='final_result', 
tool_call_id='toolu_vrtx_01SZVmMfLYHBQz8c8CP12qoP'))

[<function __main__.add(a: int, b: int) -> int>]

In [13]:
# Execute with selected tools
agent = get_agent(tools=selected_tools)
result, _ = await run_agent(agent, user_query, verbose=True)
print(result.result.output)

Agent step: UserPromptNode(user_prompt='What is the sum of 40123456789 and 2123456789?', instructions_functions=[],
system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is the sum of 40123456789 and
2123456789?', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 3, 802574, tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='I need to add these two numbers 
together. However, I notice that the available `add` function only accepts integer parameters, and these numbers 
are very large. Let me check if they fit within typical integer bounds and attempt the addition.'), 
ToolCallPart(tool_name='add', args='{"a": 40123456789, "b": 2123456789}', 
tool_call_id='toolu_vrtx_016AxYyBCK6o3zzucRgWUYRy')], usage=RequestUsage(input_tokens=590, output_tokens=122, 
details={'is_byok': False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', 
timestamp=datetime.datetime(2026, 2, 12, 13, 53, 6, 237032, tzinfo=datetime.timezone.utc), provider_name='openai', 
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'tool_calls', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 53, 3, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904383-W00F60utrrB33kSLxVyA', finish_reason='tool_call', 
run_id='8d0159f1-4f86-4433-a5d5-3b745a556d83'))

Adding 40123456789 + 2123456789


Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='add', content=42246913578, 
tool_call_id='toolu_vrtx_016AxYyBCK6o3zzucRgWUYRy', timestamp=datetime.datetime(2026, 2, 12, 13, 53, 6, 240444, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The sum of 40123456789 and 
2123456789 is **42246913578**.')], usage=RequestUsage(input_tokens=728, output_tokens=26, details={'is_byok': 
False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 
13, 53, 7, 256452, tzinfo=datetime.timezone.utc), provider_name='openai', 
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'stop', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 53, 6, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904386-XttR3R2TVLW5LuTRPXl7', finish_reason='stop', 
run_id='8d0159f1-4f86-4433-a5d5-3b745a556d83'))

Agent step: End(data=FinalResult(output='The sum of 40123456789 and 2123456789 is **42246913578**.'))

The sum of 40123456789 and 2123456789 is **42246913578**.
